In [56]:
import pandas as pd
import os


In [57]:
# Menentukan path file
file_path = "C:/Users/VSCODE/FILE/Capstone/cv-corpus-19.0-2024-09-13/id/validated_sentences.tsv"

# Menampilkan path absolut file
print("Absolute path:", os.path.abspath(file_path))

Absolute path: C:\Users\VSCODE\FILE\Capstone\cv-corpus-19.0-2024-09-13\id\validated_sentences.tsv


### Gathering Data

### Assessing Data